In [1]:
import pandas
import spotipy
import redis
from spotipy.oauth2 import SpotifyClientCredentials
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [2]:
# Set spotipy object
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id='118aa19f2b66476fbc062f0ac146d8b5',
    client_secret='7ca95a3159ab4391bee70f70d47a9271'
))

### Raccolta dati

In [44]:
search_results = []

playlists = sp.search(q='covid', type='playlist', limit=50,offset=0)['playlists']

while playlists:
    try:
        for i, playlist in enumerate(playlists['items']):
            search_results.append(playlist)
            print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
        if playlists['next']:
            print(playlists['next'])
            playlists = sp.next(playlists)['playlists']
        else:
            playlists = None
    except Exception as e:
        playlists = None
        print(e)

   1 spotify:playlist:7I5p2wl30OGnqmgrA5J0Bj COVID Sessions
   2 spotify:playlist:37i9dQZF1DWY5LGZYBBHHz Calma
   3 spotify:playlist:1ggca4HcbFPQBPTGRNPtph COVID-19 Quarantine Party
   4 spotify:playlist:2BsleOCVPKCVTuFXEb4YxH Covid 2020 mixed list
   5 spotify:playlist:5vRoiRVjHGlW6MLhpiR23Q Posada 2020 (COVID🦠)
   6 spotify:playlist:3utP2qfmAHemyyAhuOos7p COVID playlist
   7 spotify:playlist:11xZ23UurmlDUKrGkmSpJS COVID Crazies
   8 spotify:playlist:3xL8EdzgE423R8EkuhJ7pk COVID-19 Party Bangers 🤑🤑
   9 spotify:playlist:2qk6pllj9bw3pXJwvo8rE2 DANGDUT KOPLO TERBARU 2021
  10 spotify:playlist:5Sw2qVuppKXXRKCevWGzlc COVID-19 QUARANTINE PARTY PLAYLIST
  11 spotify:playlist:2GwWaID8FLRF7Gv3jUGtEl 🦠COVID Fiesta, México 🇲🇽 🍾🎈👩🏽‍💻🧑🏻‍💻
  12 spotify:playlist:3cjly3JnNA6NF3ZLfkNuyt COVID 19 SONGS
  13 spotify:playlist:4XvaaHIX80NHJruiySnuEt SESIÓN VERMÚ COVID-19
  14 spotify:playlist:0Etgp7mzu27vG7k2RJwUYP COVID feelz or just feels 
  15 spotify:playlist:2GhesYT8cbez9JAcMkywpv Fin De Año 2020! A

 151 spotify:playlist:4yF4YSmJK9xPTnspWQygKN covid loner
 152 spotify:playlist:4yuAS0YySUoyZW7uumHGNQ Sala Radar Vigo (Navidad 2020 #Covid)
 153 spotify:playlist:4KxyzLpLhSdxjus7IfDMho YASUKO COVID
 154 spotify:playlist:5X2upELn1nMyCvmybs6Egl F*CK COVID
 155 spotify:playlist:144lgzNr3jCGmvtXhTVtVM covid-19 bangers
 156 spotify:playlist:0jNpnVEZzANiEPD2gMZZhr Kabbalat Shabbat COVID-19
 157 spotify:playlist:2U4J1vAVC8DvV3fPfrgXVy Plenas que no pudimos bailar por COVID
 158 spotify:playlist:1ltYTuRJ6rKjz0h3AKvHzT Pandemia COVID 
 159 spotify:playlist:5yaqmNGr9gSC5Uavwc3zFc Music for COVID
 160 spotify:playlist:60OeWep1D9rYVGdSSFZycl COVID-19 Cuarentena Party Musica de Antro Circuit Gay 2020 - 2021
 161 spotify:playlist:1ri9iDMblW3MLIBCRJJ2sM 14 horas de perreo tan sucio que te da Covid-19
 162 spotify:playlist:1hTw0Q9qEu75O5LzLoOKUA bedroom dance party 
 163 spotify:playlist:62pk09JHUKly9GJhoREvxC Covid😷🤯
 164 spotify:playlist:7mEyZ2qiOKpFunpjbJznYU PedritoVm - COVID-19 PLAYLIST
 165 spot

 301 spotify:playlist:2itP43PQxbDKGe4sBDd8gt COVID 19 
 302 spotify:playlist:0XsbHxXPdfPXGvHg0uwV2J BenMusic Fucks the Covid
 303 spotify:playlist:1mKHaD9yxir05FYg35Fqf1 Covid free
 304 spotify:playlist:5SOf7xBXjBdJzJCGAwaJ6z Verano 2020 COVID CHÚPAME LA PIJA 
 305 spotify:playlist:0coHa0O3n4cBbp9uaP4zYB COVID-19
 306 spotify:playlist:2MTdgixcBzqSHvCv8r5Gte 🌡🥵Covid🦠🧪
 307 spotify:playlist:0ZHifvHs5yxBArHiaih0yb COVID-19
 308 spotify:playlist:1PkL1q39t5ZzarTj98HFxx Fuck covid🖕
 309 spotify:playlist:7INwWVzVuHlOBux5dBTjMV Covid
 310 spotify:playlist:1js7jSUYCqvoX9QlI83V93 Full Coverage Fridays
 311 spotify:playlist:3sxWXtLNYFbNYar5qMStvg CHAO COVID 🚫🦠
 312 spotify:playlist:3ipj5pImRqqCGGgoNN8jkV COVID2020
 313 spotify:playlist:1Vk305JwJw8usgpA6xigO0 Covid country 
 314 spotify:playlist:5TUn6pFPaDL5MXCDF2geav covid-19 
 315 spotify:playlist:1OmtYDlnRNfZI1CWFagB9P COVID19
 316 spotify:playlist:07dz85efApenICz2uYmDCb Heal the world- Covid19 tribute
 317 spotify:playlist:2cr6DWG8XhWdTYOkwpzQ

 451 spotify:playlist:0IbsuNpzQ1SgJLo6I9u3dv covid
 452 spotify:playlist:0kZ08YlfRSton43dlMJNue COVID-420
 453 spotify:playlist:5ZY0h3tAGEdFounoeAI4aI COVID Appropriate 
 454 spotify:playlist:28wirnzDDTW826NxL3moK9 Covid workout
 455 spotify:playlist:50RfgtG1ufOZMLF3fqLkzL CovidHustle.mp3
 456 spotify:playlist:3BCmr8YIT192EC4lkslj7v Covid 19
 457 spotify:playlist:6Yq7c2gIbthgFC9mCqLfGN Halloweekend 2020 (Covid Edition)
 458 spotify:playlist:0lTjJV5UZpLdyXmmOZxyST COVID
 459 spotify:playlist:2yTmeL2FmPMO9TlGBxdK0F COVID 2.0
 460 spotify:playlist:6EUIOmNzKvAQpeEHSsufCc COVID 2021
 461 spotify:playlist:0H46vltn4HoppNTHVraILd covid tunes
 462 spotify:playlist:1RLW8vuNEeLf4tZxwQ6z0P Covid-19 sleep
 463 spotify:playlist:0KrnX3jR4IucxkS4BoBXg9 covid19
 464 spotify:playlist:1fvYRn2S9FxFpEqfyKUa3N Covid-999
 465 spotify:playlist:2KDa6BAV7S7bxNLcUei3U6 Covid1
 466 spotify:playlist:58N2Uhc85Hev1Cu0q8mH4R France2021<3 fuck covid
 467 spotify:playlist:4N7pJ3EFSk4UHOPAwMXBH6 2 O 2 O  - Covid 🦠🔥
 468

 601 spotify:playlist:7oCJ2IAsh9ttgfqSinKjSw Covid
 602 spotify:playlist:2UcWIxErYMRYyTFCBKcbok Covid Hangs
 603 spotify:playlist:56imomShb1E6kqi7vQ2Lrb Covid-19
 604 spotify:playlist:4iQKuRisTvHa9OtAN31FoJ COVID-19 ERA
 605 spotify:playlist:7cS3KMsESJBnqnt3ZK030r Covideren
 606 spotify:playlist:6sCugiLUgLPF9OHv0lfheA Covid-19🏞🌄
 607 spotify:playlist:7dlAqeG2ZilvCwvKf9ggXI Covid 19
 608 spotify:playlist:7fAo0TrJOnh1yXjGMMBJdi Covid20x 
 609 spotify:playlist:5bZEMIFzybD7c28wb5QDkz perreo en covid 
 610 spotify:playlist:3euFDbZ6bAd29BkP7flGAO Covid Bngsd
 611 spotify:playlist:1tfpyvk2IrMENESptJok4V Baby it's Covid Outside
 612 spotify:playlist:3Ic1r9JTTOiMygMraE7Od5  COVID-19 2.0
 613 spotify:playlist:2taNy0mManzRHhJIjBn00l Covid Party
 614 spotify:playlist:6atW4OylDrFUYpH3kboNwn COVID 19
 615 spotify:playlist:7aTlAxjwP0tMb1C7Bm3kye Covid free
 616 spotify:playlist:0S9HKKPyGkK4tzCbEhrNbv TIHHY COVID ON THAT BEAT 
 617 spotify:playlist:5LGqhtQHQJX1nvV0SQAlVd Lollapalooza Brasil 2020 #Loll

...

HTTP Error for GET to https://api.spotify.com/v1/search?query=covid&type=playlist&offset=2000&limit=50 returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search?query=covid&type=playlist&offset=2000&limit=50 returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search?query=covid&type=playlist&offset=2000&limit=50 returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search?query=covid&type=playlist&offset=2000&limit=50 returned 404 due to Not found.


http status: 404, code:-1 - https://api.spotify.com/v1/search?query=covid&type=playlist&offset=2000&limit=50:
 Not found., reason: None
1951 spotify:playlist:7by6OMPjDmum1aszrBeThr COVID christmas 2020
1952 spotify:playlist:1lYxqWXPSoBFCpp2RMev8z covid
1953 spotify:playlist:6rPbTnmtKkg0AmqVJZkKbG COVID-19 EDM
1954 spotify:playlist:23jlE0NZRRQmhPNKIoSLHu COVID OG
1955 spotify:playlist:1XswOj4mHgTT3KJek069JM Covid-19
1956 spotify:playlist:2dg4Pcdk2qhbYewwAFabqL COVID
1957 spotify:playlist:4TwDVKDVkVUc5F6SXHOU11 A very covid winter
1958 spotify:playlist:5oRUJ6zADHmGQE8IV7iEeL CovidSummer
1959 spotify:playlist:3M8ZCoTePTCBwXBbqeakqc Covid
1960 spotify:playlist:6LBSRQgzgHFCYvRo2P1Etv COVID
1961 spotify:playlist:4VxRxmBfWX1Ii9Cr3otQqa Covid-19
1962 spotify:playlist:28yFPWa7JwW2ObHguE1WcH Covid
1963 spotify:playlist:76NknbrSMv0UNDIfq0qqxN Covid tunes
1964 spotify:playlist:4RoF9lWCs4eLqWztwNaFee 202covid
1965 spotify:playlist:4UfiOyJpbu0gIbiNZilals COVID Quarantine
1966 spotify:playlist:4ws3vM

HTTP Error for GET to https://api.spotify.com/v1/search?query=covid&type=playlist&offset=2000&limit=50 returned 404 due to Not found.


http status: 404, code:-1 - https://api.spotify.com/v1/search?query=covid&type=playlist&offset=2000&limit=50:
 Not found., reason: None
1951 spotify:playlist:7by6OMPjDmum1aszrBeThr COVID christmas 2020
1952 spotify:playlist:1lYxqWXPSoBFCpp2RMev8z covid
1953 spotify:playlist:6rPbTnmtKkg0AmqVJZkKbG COVID-19 EDM
1954 spotify:playlist:23jlE0NZRRQmhPNKIoSLHu COVID OG
1955 spotify:playlist:1XswOj4mHgTT3KJek069JM Covid-19
1956 spotify:playlist:2dg4Pcdk2qhbYewwAFabqL COVID
1957 spotify:playlist:4TwDVKDVkVUc5F6SXHOU11 A very covid winter
1958 spotify:playlist:5oRUJ6zADHmGQE8IV7iEeL CovidSummer
1959 spotify:playlist:3M8ZCoTePTCBwXBbqeakqc Covid
1960 spotify:playlist:6LBSRQgzgHFCYvRo2P1Etv COVID
1961 spotify:playlist:4VxRxmBfWX1Ii9Cr3otQqa Covid-19
1962 spotify:playlist:28yFPWa7JwW2ObHguE1WcH Covid
1963 spotify:playlist:76NknbrSMv0UNDIfq0qqxN Covid tunes
1964 spotify:playlist:4RoF9lWCs4eLqWztwNaFee 202covid
1965 spotify:playlist:4UfiOyJpbu0gIbiNZilals COVID Quarantine
1966 spotify:playlist:4ws3vM

KeyboardInterrupt: 

In [4]:
sp.market(search_results[0]['uri'])

AttributeError: 'Spotify' object has no attribute 'market'

In [45]:
final_informations = []

for playlist in search_results:
    try:
        tracks = sp.playlist(playlist['uri'])['tracks']['items']
        track_information = []
        for track in tracks:
            track_information.append({
                "track_id": track['track']['id'],
                "available_markets" : ','.join(track['track']['available_markets']),
            })
        final_informations.append({
            "playlist_id": playlist['id'],
            "tracks": json.dumps(track_information)
        })
    except Exception as e:
        print(e)
len(final_informations)


'NoneType' object is not subscriptable


KeyboardInterrupt: 

In [6]:
len(final_informations)

1930

In [7]:
# Connetto a redis e lancio un ping
REDIS_HOST = "jupyter_redis"
REDIS_PORT = 6379
REDIS = redis.Redis(host=REDIS_HOST, port=REDIS_PORT)
# test the connection
REDIS.ping()

True

In [30]:
for i,playlist in enumerate(final_informations):
    REDIS.set(str(i),json.dumps(playlist))
    

In [60]:
json.loads(REDIS.get(1))['tracks'][1]

'{'

# Secondo file 
Carico le keys da redis e le aumento con i dati riguardanti le audio features.

In [72]:
for index in REDIS.keys('*'):
    tracks = json.loads(json.loads(REDIS.get(int(index)))['tracks'])
    for track in tracks:
        track['track_id']
    break

0xXhgLXfEHnWJHVgRvFvdn
4S6gJt4RYR34t4sjW8wEah
3gPOFPYLgpvQz0ZMwmv7NE
3RD0PGolM0mETGHaprwgMn
7gDh37dQIzKIFMUUnwPNj4
7IwNPHjpZMkggv0qlh1rUO
7vhesMGuUU7By3RWmS73wO
2Wc6NRKeolElmwTTS6yQ4V
1Pc4HQCKJbvDoCdC0TqR4P
6w145sHWdykyab2D6sE5wl
59f765mlM08j2POCl3zPvd
1wh8lVW8wvVnlV18BoyhvZ
5qd8Q5J3mjBkWtPrbIhorO
4iCJFRTRsFlj4UWoEd7bS9
2NZ4Lg677xRbdrlFMOPTcF
04cd1MHVq1D8v1nuTZoCdv
2EzRdhuoPyNJDlep7lnNf2
0Myz4UuooJeosNmbscdfZZ
4bVaNRaJSYyD0Z2dde42I0
5KgdtjmglXPjJNKvMvSjgo
71iAechwVKGu26pahzFL0k
6z1wOs7wdx5HSn6PMDoc4s
2GSr8anleCXKLNjjjijoMZ
2vCHDrXVN0k1GPTjSIZ69E
6gBrTZLAO7tLl7N7kg8fNX
4W6za66pgH7QBbeiU9kS1v
62clthR0fZmLbxeqxVJatQ
70c9sdkTnDiehSDOCxCnDc
5HFOgL611IRiarTnoKyP4C
00eP3Re9E5u29vdU0oPuRK
1zHQ5gjJoNxT4RYFmGEWuo
3y5ahYM9zwyOCt3mieZ0PZ
7hbgCKddEQhCVXHsMdHebN
28Ct4qwkQXY2W5yyNCLuVI
2bvO1KXbUd4vySCWGNqIIa
4m1fxkdxv39jPicSP4kYG5
53e9kK5U7udi2SqDzaEIVv
2J4qy8RyfwgXHt73cWOE6P
6QMgs72iwsAuEPobHbH8Jf
28upjDakWVnHAgW2anm273
3T0OdyDjFtltn1nHH4j06U
5kIcrM3QVD4BQPFMszQnU1
4waqcUQWdj0yH26STWl2Rq
37miw1ff4zH

In [5]:
len(final_informations)

1931

In [50]:
sp.audio_features(tracks=["5sXPAGXZEnl7dMReMWwFTg","5sXPAGXZEnl7dMReMWwFTg"])

[{'danceability': 0.796,
  'energy': 0.419,
  'key': 2,
  'loudness': -9.686,
  'mode': 1,
  'speechiness': 0.0704,
  'acousticness': 0.114,
  'instrumentalness': 0.938,
  'liveness': 0.105,
  'valence': 0.165,
  'tempo': 94.958,
  'type': 'audio_features',
  'id': '5sXPAGXZEnl7dMReMWwFTg',
  'uri': 'spotify:track:5sXPAGXZEnl7dMReMWwFTg',
  'track_href': 'https://api.spotify.com/v1/tracks/5sXPAGXZEnl7dMReMWwFTg',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5sXPAGXZEnl7dMReMWwFTg',
  'duration_ms': 205547,
  'time_signature': 4},
 {'danceability': 0.796,
  'energy': 0.419,
  'key': 2,
  'loudness': -9.686,
  'mode': 1,
  'speechiness': 0.0704,
  'acousticness': 0.114,
  'instrumentalness': 0.938,
  'liveness': 0.105,
  'valence': 0.165,
  'tempo': 94.958,
  'type': 'audio_features',
  'id': '5sXPAGXZEnl7dMReMWwFTg',
  'uri': 'spotify:track:5sXPAGXZEnl7dMReMWwFTg',
  'track_href': 'https://api.spotify.com/v1/tracks/5sXPAGXZEnl7dMReMWwFTg',
  'analysis_url': 'https://api.

Lunghezza items: 75136
### Connessione REDIS

In [7]:
#
# connect to redis running on this machine on its default port 6379
#
REDIS_HOST = "jupyter_redis"
REDIS_PORT = 6379
REDIS = redis.Redis(host=REDIS_HOST, port=REDIS_PORT)
# test the connection
REDIS.ping()

True

In [12]:
for i, playlist in enumerate(final_informations):
    REDIS.hset(name = "playlist:{}".format(i),key = i, value=playlist)

DataError: Invalid input of type: 'dict'. Convert to a bytes, string, int or float first.

In [47]:
len(REDIS.keys("*"))

1931

## Connessione postgres 

In [77]:
import psycopg2

# connect to database
connect_str = 'host=postgres port=5432 dbname=spotify user=postgres password=postgres1234'
conn = psycopg2.connect(connect_str)
conn.autocommit = True
cursor = conn.cursor()

In [78]:
cursor

<cursor object at 0x7f0a301d05e0; closed: 0>

### Connessione POSTGRES da spark

In [73]:
spark = SparkSession \
    .builder \
    .appName('04_notebook') \
    .config('spark.driver.extraClassPath', 'postgresql-42.2.10.jar') \
    .getOrCreate()

In [74]:
properties = {
    'driver': 'org.postgresql.Driver',
    'url': 'jdbc:postgresql://postgres:5432/spotify',
    'user': 'postgres',
    'password': 'postgres1234',
    'dbtable': ' spotify_details',
}

In [75]:
df2 = spark.read \
    .format('jdbc') \
    .option('driver', properties['driver']) \
    .option('url', properties['url']) \
    .option('user', properties['user']) \
    .option('password', properties['password']) \
    .option('dbtable', properties['dbtable']) \
    .load()

In [76]:
df2.show(10, False)

+-----------+--------+-----------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+-----------+--------------+
|playlist_id|track_id|available_markets|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo|duration_ms|time_signature|
+-----------+--------+-----------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+-----------+--------------+
+-----------+--------+-----------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+-----------+--------------+

